In [1]:
import os
import pandas as pd

import preprocessing as util

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='bs4')

In [2]:
# Path
cwd = os.getcwd()
csv_path = os.path.join(cwd, 'data/csv/')

data_files = ['dataset_1.csv', 'dataset_2.csv']

In [3]:
dataset_1 = pd.read_csv(os.path.join(csv_path, data_files[0]), index_col=0, encoding='latin-1', dtype={'Body': 'object', 'Class': 'bool'})
dataset_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3440 entries, 0 to 3439
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Body    3440 non-null   object
 1   Class   3440 non-null   bool  
dtypes: bool(1), object(1)
memory usage: 57.1+ KB


In [4]:
dataset_2 = pd.read_csv(os.path.join(csv_path, data_files[1]), index_col=0, encoding='latin-1', dtype={'Body': 'object', 'Class': 'bool'})
dataset_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18920 entries, 0 to 18919
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Body    18920 non-null  object
 1   Class   18920 non-null  bool  
dtypes: bool(1), object(1)
memory usage: 314.1+ KB


## Preprocessing

We need to convert the text data into a format more suitable for use with machine learning algorithms.<br>
The process will follow the steps below:

### 1. Cleanup HTML and whitespace

A percentage of the emails extracted are either in HTML format or they have the same message in both plaintext and HTML, as part of a `multipart/alternative` content-type message.<br>
In order to extract the text, the HTML formatting has to be removed, along with unnecessary whitespace any duplicated text created by the aforementioned multipart emails.

In [5]:
dataset_1['Body'] = dataset_1['Body'].apply(util.strip_characters)
dataset_1['Body'] = dataset_1['Body'].apply(util.deduplicate_text)

In [6]:
dataset_2['Body'] = dataset_2['Body'].apply(util.strip_characters)
dataset_2['Body'] = dataset_2['Body'].apply(util.deduplicate_text)